In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from pathlib import Path

notebook_path = Path().absolute()
sys.path.append(str(notebook_path.parent))

In [3]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, MllamaForConditionalGeneration, AutoProcessor 
# from transformers import BitsAndBytesConfig, Gemma3ForCausalLM
# from janus.models import MultiModalityCausalLM, VLChatProcessor
from utils import harmful_dataset
from neural_controllers import NeuralController
from utils import LLMType
from collections import namedtuple 
import json
import requests
from PIL import Image
import io
from tqdm import tqdm 
import pickle
import gc

# from janus.utils.io import load_pil_images
# from generation_utils import extract_image
import utils

from inversion_utils import *
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# from transformers import logging
# logging.set_verbosity_error() # Only show errors

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)


In [4]:
torch.backends.cudnn.benchmark = True 
torch.backends.cuda.matmul.allow_tf32 = True

LLM = namedtuple('LLM', ['language_model', 'tokenizer', 'processor', 'name', 'model_type'])

In [5]:
def read_file(fname, lower=True):

    concepts = []
    with open(fname, encoding="utf-8") as f: 
        for line in f:
            if lower:
                concepts.append(line.strip().lower())
            else:
                concepts.append(line.strip())
    concepts = sorted(list(set(concepts)))
    return concepts


def select_llm(model_type, MODEL_VERSION='3.1', MODEL_SIZE='8B'):

    custom_cache_dir = "/scratch/bbjr/skarmakar/huggingface"

    if model_type=='llama':

        if MODEL_VERSION == '3.1' and MODEL_SIZE == '8B':
            model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
        elif MODEL_VERSION == '3.1' and MODEL_SIZE == '70B':
            model_id = "unsloth/Meta-Llama-3.1-70B-Instruct-bnb-4bit"
        elif MODEL_VERSION == '3.3' and MODEL_SIZE == '70B':
            model_id = "unsloth/Llama-3.3-70B-Instruct-bnb-4bit"

        language_model = AutoModelForCausalLM.from_pretrained(
            model_id, device_map="cuda", cache_dir=custom_cache_dir,
        )

        use_fast_tokenizer = "LlamaForCausalLM" not in language_model.config.architectures
        tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=use_fast_tokenizer, padding_side="left", legacy=False)
        tokenizer.pad_token_id = 0 
        # model_name='llama_3_8b_it'
        if MODEL_VERSION == '3.1' and MODEL_SIZE == '8B':
            model_name='llama_3_8b_it_eng_only'
        elif MODEL_VERSION == '3.1' and MODEL_SIZE == '70B':
            model_name = "llama_3.1_70b_it_eng_only"
        elif MODEL_VERSION == '3.3' and MODEL_SIZE == '70B':
            model_name = "llama_3.3_70b_it_eng_only"

        processor = None
        llm_type = LLMType.TEXT

        language_model.generation_config.pad_token_id = tokenizer.pad_token_id # to disable the warning

        language_model.generation_config.temperature=None # to disable the stupid warnings
        language_model.generation_config.top_p=None # to disable the stupid warnings
        # language_model.generation_config.top_k=None # to disable the stupid warnings

    elif model_type=='gemma':
        quantization_config = BitsAndBytesConfig(load_in_8bit=True)
        if MODEL_VERSION == '3.1' and MODEL_SIZE == '1B':
            model_id = "google/gemma-3-1b-it"

        tokenizer = AutoTokenizer.from_pretrained(model_id)

        language_model = Gemma3ForCausalLM.from_pretrained(
            model_id, quantization_config=quantization_config
        ).eval()

        if MODEL_VERSION == '3.1' and MODEL_SIZE == '1B':
            model_name='gemma_3_1b_it_eng_only'

        processor = None
        llm_type = LLMType.GEMMA_TEXT

        # print(tokenizer.chat_template)

    llm = LLM(language_model, tokenizer, processor, model_name, llm_type)
    # print(llm.language_model)
    return llm


def test_concept_vector(controller, concept="___", prompts=[], coef=0.75, max_tokens=100, orig=True, image=None):

    # outputs = []
    for prompt in prompts:
        if orig:
            print("\n========================== No Control ==========================")
            original_output = controller.generate(prompt, image=image, max_new_tokens=max_tokens, do_sample=False)#, temperature=0)
            print(original_output)

        print(f"\n========================== + {concept} Control (normal) ==========================")
        steered_output = controller.generate(prompt,
                                            image=image, 
                                            layers_to_control=controller.hidden_layers,
                                            control_coef=coef,
                                            max_new_tokens=max_tokens,
                                            do_sample=False)
        print(steered_output)

        torch.cuda.empty_cache()

    # return outputs


def load_controller(llm, concept, path='../directions_moods/'):
    tcontroller = NeuralController(
        llm,
        llm.tokenizer,
        rfm_iters=8,
        control_method="rfm",
        n_components=1,
    )

    tcontroller.load(concept=concept, model_name=llm.name, path=path)

    return tcontroller


def compare_pearson(concept1, concept2):
    total_v = 0
    for l in concept1:
        mat = torch.stack((concept1[l][0], concept2[l][0]))
        
        pearson = torch.corrcoef(mat)
        v = pearson[0, 1].item()
        print(f'layer: {l}, PCC: {v}')
        total_v += v
    
    print(f'total: {total_v}')
    return total_v


def compare_cosine(concept1, concept2):
    total_c = 0
    for l in concept1:
        cosine = torch.nn.functional.cosine_similarity(concept1[l][0], concept2[l][0], dim=0)
        c = cosine.item()

        print(f'layer: {l}, cosine: {c}')
        total_c += c
        
    print(f'total: {total_c}')
    return total_c


def just_dirs(llm, concept, path='../directions_moods/'):
    tcontroller = NeuralController(
        llm,
        llm.tokenizer,
        rfm_iters=8,
        control_method="rfm",
        n_components=1
    )

    tcontroller.load(concept=concept, model_name=llm.name, path=path)

    return tcontroller.directions

In [6]:
model_type = 'llama'
MODEL_VERSION = '3.1'
# MODEL_SIZE = '70B'
MODEL_SIZE = '8B'
llm = select_llm(model_type, MODEL_VERSION=MODEL_VERSION, MODEL_SIZE=MODEL_SIZE)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
hidden_layers = range(-1,-32,-1)

In [22]:
# Average vector

fname = '../data/moods/moods_antonyms.txt'
lower = True
dataset_label = 'moods'

concepts = read_file(fname, lower=lower)

all_dirs = {i: [] for i in hidden_layers}

for concept_idx, concept in enumerate(tqdm(concepts)):
    dirs = just_dirs(llm, concept)
    for i in hidden_layers:
        all_dirs[i].append(dirs[i][0])


t_all_dirs = {i: torch.stack(all_dirs[i]) for i in all_dirs}



  0%|          | 0/478 [00:00<?, ?it/s]/u/skarmakar1/miniconda3/envs/neucon/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

D

 14%|█▍        | 66/478 [00:00<00:02, 157.57it/s]

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components

 23%|██▎       | 110/478 [00:00<00:01, 189.91it/s]

...

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components

 92%|█████████▏| 439/478 [00:02<00:00, 197.24it/s]

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components

100%|██████████| 478/478 [00:02<00:00, 186.25it/s]

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components

In [25]:
from scipy.linalg import orth

In [56]:
print(t_all_dirs[-1].shape)

torch.Size([478, 4096])


In [57]:
for i in t_all_dirs:
    vectors_np = t_all_dirs[i].T.cpu().numpy()

    # orthonormal_basis = orth(vectors_np, rcond=1e-3)
    orthonormal_basis = orth(vectors_np, rcond=1e-2)
    # orthonormal_basis = orth(vectors_np, rcond=1e-1)

    print(f"Layer: {i}, Basis shape: {orthonormal_basis.shape}")

Layer: -1, Basis shape: (4096, 470)
Layer: -2, Basis shape: (4096, 471)
Layer: -3, Basis shape: (4096, 470)
Layer: -4, Basis shape: (4096, 470)
Layer: -5, Basis shape: (4096, 466)
Layer: -6, Basis shape: (4096, 461)
Layer: -7, Basis shape: (4096, 455)
Layer: -8, Basis shape: (4096, 445)
Layer: -9, Basis shape: (4096, 436)
Layer: -10, Basis shape: (4096, 425)
Layer: -11, Basis shape: (4096, 418)
Layer: -12, Basis shape: (4096, 399)
Layer: -13, Basis shape: (4096, 384)
Layer: -14, Basis shape: (4096, 373)
Layer: -15, Basis shape: (4096, 362)
Layer: -16, Basis shape: (4096, 341)
Layer: -17, Basis shape: (4096, 315)
Layer: -18, Basis shape: (4096, 276)
Layer: -19, Basis shape: (4096, 213)
Layer: -20, Basis shape: (4096, 174)
Layer: -21, Basis shape: (4096, 179)
Layer: -22, Basis shape: (4096, 162)
Layer: -23, Basis shape: (4096, 143)
Layer: -24, Basis shape: (4096, 116)
Layer: -25, Basis shape: (4096, 85)
Layer: -26, Basis shape: (4096, 94)
Layer: -27, Basis shape: (4096, 76)
Layer: -28, B